In [1]:
import json
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle

from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import recall_score, accuracy_score, f1_score, auc, roc_auc_score, precision_score, balanced_accuracy_score, fbeta_score, make_scorer
from sklearn.inspection import permutation_importance

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.svm import SVC

from imblearn.over_sampling import RandomOverSampler, ADASYN, SMOTE
from imblearn.under_sampling import RandomUnderSampler


# Baseline

In [2]:
def get_baseline(json_content):

    baseline_stats = []
    baseline_inds = []

    for run in json_content['runs']:
        
        config = run['configuration']
        source_files = config['source_files']
        samplingrates = config['sampling_rates']
        
        samplingmethods = config['sampling_methods']
        is_baseline = config['is_baseline']
        
        column_statistics = run['column_statistics']
        for col_stats in column_statistics:
        
            table_name = col_stats['column_information']['table_name']
            column_name = col_stats['column_information']['column_name']
            count = col_stats['count']
            unique_count = col_stats['unique_count']
            unique_ratio = col_stats['unique_ratio']
            
            if is_baseline:
                baseline_stats.append([table_name, column_name, count, unique_count, unique_ratio, 1, 'None'])
        
        results = run['results']
        for result in results['inds']:
            dependent = result['dependents'][0]
            dependent_table_name = dependent['table_name']
            dependent_column_name = dependent['column_name']
            referenced = result['referenced'][0]
            referenced_table_name = referenced['table_name']
            referenced_column_name = referenced['column_name']
            
            if is_baseline:
                baseline_inds.append([dependent_table_name, dependent_column_name, referenced_table_name, referenced_column_name])
    baseline_df = pd.DataFrame(baseline_stats, columns=['table_name', 'column_name', 'count', 'unique_count', 'unique_ratio', 'sampling_rate', 'sampling_method'])

    return baseline_df, baseline_inds



In [3]:
def get_run_stats(json_content, baseline_df, baseline_inds):
    sampled_stats = []
    ind_dfs = []

    labels = []

    for run in json_content['runs']:
        
        config = run['configuration']
        source_files = config['source_files']
        samplingrates = config['sampling_rates']
        
        samplingmethods = config['sampling_methods']
        is_baseline = config['is_baseline']
        if is_baseline:
            continue
        
        run_stats = []
        column_statistics = run['column_statistics']
        for col_stats in column_statistics:
        
            table_name = col_stats['column_information']['table_name']
            column_name = col_stats['column_information']['column_name']
            count = col_stats['count']
            unique_count = col_stats['unique_count']
            unique_ratio = col_stats['unique_ratio']
            
            if is_baseline:
                run_stats.append([table_name, column_name, count, unique_count, unique_ratio, 1, 'None'])
            else:
                run_stats.append([table_name, column_name, count, unique_count, unique_ratio, samplingrates[0], samplingmethods[0]])
        
        sampled_stats.extend(run_stats)
        stats_df = pd.DataFrame(run_stats, columns=['table_name', 'column_name', 'count', 'unique_count', 'unique_ratio', 'sampling_rate', 'sampling_method'])
        
        run_inds = []
        
        results = run['results']
        for result in results['inds']:
            dependent = result['dependents'][0]
            dependent_table_name = dependent['table_name']
            dependent_column_name = dependent['column_name']
            referenced = result['referenced'][0]
            referenced_table_name = referenced['table_name']
            referenced_column_name = referenced['column_name']
            
            if is_baseline:
                labels.append(1)
            else:
                if [dependent_table_name, dependent_column_name, referenced_table_name, referenced_column_name] in baseline_inds:
                    labels.append(1)
                else:
                    labels.append(0)
                    
            missing_values = result['errors'][0]['missing_values']
        
            run_inds.append([dependent_table_name, dependent_column_name, referenced_table_name, referenced_column_name, missing_values])
        
        
        # baseline_df = pd.DataFrame(baseline_stats, columns=['table_name', 'column_name', 'count', 'unique_count', 'unique_ratio', 'sampling_rate', 'sampling_method'])
        run_df = pd.DataFrame(run_inds, columns=['left_table', 'left_column', 'right_table', 'right_column', 'missing_values'])
        
        # Merge with Baseline Stats
        # Left
        merged_df = pd.merge(run_df, baseline_df, left_on=['left_table', 'left_column'], right_on=['table_name', 'column_name'])
        merged_df.drop(['table_name', 'column_name', 'unique_count', 'sampling_rate', 'sampling_method'], axis='columns', inplace=True)
        merged_df.rename({'count': 'left_baseline_count', 
                        'unique_ratio': 'left_baseline_unique_ratio',
                        'sampling_rate': 'left_baseline_sampling_rate'}, 
                        axis='columns',
                        inplace=True)
        # Right
        merged_df = pd.merge(merged_df, baseline_df, left_on=['right_table', 'right_column'], right_on=['table_name', 'column_name'])
        merged_df.drop(['table_name', 'column_name', 'unique_count', 'sampling_rate', 'sampling_method'], axis='columns', inplace=True)
        merged_df.rename({'count': 'right_baseline_count', 
                        'unique_ratio': 'right_baseline_unique_ratio',
                        'sampling_rate': 'right_baseline_sampling_rate'}, 
                        axis='columns',
                        inplace=True)
        
        # Merge with sampled stats
        # Left
        merged_df = pd.merge(merged_df, stats_df, left_on=['left_table', 'left_column'], right_on=['table_name', 'column_name'])
        merged_df.drop(['table_name', 'column_name', 'unique_count'], axis='columns', inplace=True)
        merged_df.rename({'count': 'left_count', 
                        'unique_ratio': 'left_unique_ratio',
                        'sampling_rate': 'left_sampling_rate',
                        'sampling_method': 'left_sampling_method'}, 
                        axis='columns',
                        inplace=True)
        # Right
        merged_df = pd.merge(merged_df, stats_df, left_on=['right_table', 'right_column'], right_on=['table_name', 'column_name'])
        merged_df.drop(['table_name', 'column_name', 'unique_count'], axis='columns', inplace=True)
        merged_df.rename({'count': 'right_count', 
                        'unique_ratio': 'right_unique_ratio',
                        'sampling_rate': 'right_sampling_rate',
                        'sampling_method': 'right_sampling_method'}, 
                        axis='columns',
                        inplace=True)
        ind_dfs.append(merged_df)
        
    # sampled_stats_df = pd.DataFrame(sampled_stats, columns=['table_name', 'column_name', 'count', 'unique_count', 'unique_ratio', 'sampling_rate', 'sampling_method'])

    features = pd.concat(ind_dfs)
    features.drop(['right_table', 'right_column', 'left_table', 'left_column'], axis='columns', inplace=True)
    
    labels = pd.DataFrame(labels, columns=['labels'])
    
    return features, labels
    

## Feature Engineering

In [4]:
def ohe(features):
    ohe_cols = features['left_sampling_method'].to_numpy().reshape(-1, 1)
    ohe = OneHotEncoder(sparse_output=False).fit(ohe_cols)
    ohe_df = ohe.transform(ohe_cols)

    ohe_df = pd.DataFrame(ohe_df, columns=ohe.get_feature_names_out())

    features.drop(['left_sampling_method'], axis='columns', inplace=True)
    features = pd.concat([features.reset_index().reindex(ohe_df.index), ohe_df], axis=1)
    return features


def drop_useless_features(features):
    # return features.drop(['left_sampling_method', 'right_sampling_method', 'left_unique_ratio', 'right_unique_ratio'], axis='columns')
    return features.drop(['right_sampling_method', 'left_unique_ratio', 'right_unique_ratio'], axis='columns')

## Combine Everything

In [5]:
def get_features_labels(json_content):
    baseline_df, baseline_inds = get_baseline(json_content)
    features, labels = get_run_stats(json_content, baseline_df, baseline_inds)
    features = drop_useless_features(features)
    
    features['cardinality_ratio'] = features['left_baseline_count'] / features['right_baseline_count']
    features['sample_size_ratio'] = features['left_count'] / features['right_count']
    # features['missing_ratio'] = features['missing_values'] / features['left_count']
    # features['useless_ratio'] = features['missing_values'] / features['right_count']
    
    return features, labels
    
def get_features_labels_v2(json_content):
    baseline_df, baseline_inds = get_baseline(json_content)
    features, labels = get_run_stats(json_content, baseline_df, baseline_inds)
    return features, labels

## Iterate over all JSONs in current dir

In [6]:
features, labels = [], []

for f in os.listdir('.'):
    if os.path.isfile(f):
        if 'json' in f:
            print(f)
            with open(f) as file:
                json_content = json.load(file)
                
                X, y = get_features_labels(json_content)
                features.append(X)
                labels.append(y)

features = pd.concat(features)
features = ohe(features)
features.drop(['index'], axis='columns', inplace=True)
# Scaling features
features = pd.DataFrame(MaxAbsScaler().fit_transform(features), columns=features.columns)

# column_titles = ['missing_values', 
#  'left_count', 'left_sampling_rate', 'left_baseline_count', 'left_baseline_unique_ratio',
#  'right_count', 'right_sampling_rate', 'right_baseline_count', 'right_baseline_unique_ratio']

# features = features.reindex(columns=column_titles)

labels = pd.concat(labels)

features.to_csv('features.csv')
labels.to_csv('labels.csv')


X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=1, stratify=labels)

y_train = y_train.to_numpy().ravel()
y_test = y_test.to_numpy().ravel()

data_CENSUS.json
data_ENSEMBL.json
data_COMA.json
data_TESMA.json
data_CATH.json
data_TPCH.json
data_SCOP.json


In [7]:
print(features.shape)
print(labels.value_counts())
print(features.columns)

(518102, 17)
labels
0         507375
1          10727
dtype: int64
Index(['missing_values', 'left_baseline_count', 'left_baseline_unique_ratio',
       'right_baseline_count', 'right_baseline_unique_ratio', 'left_count',
       'left_sampling_rate', 'right_count', 'right_sampling_rate',
       'cardinality_ratio', 'sample_size_ratio', 'x0_biggest-value',
       'x0_evenly-spaced', 'x0_first', 'x0_longest-value', 'x0_random',
       'x0_smallest-value'],
      dtype='object')


In [8]:
features.to_csv('features.csv')
labels.to_csv('labels.csv')

In [8]:
# features.head()

In [9]:
# features.corr()

In [10]:
# features.cov()